In [1]:

import torch
import pandas as pd
import shelve
import features
import nnue_dataset
from tqdm import tqdm
import logging 
import numpy as np
from scipy.sparse import lil_matrix, vstack
import click
import os
import pickle as pkl
import model_single_bucket as M


def get_data(feature_set, fens):
    b = nnue_dataset.make_sparse_batch_from_fens(
        feature_set, fens, [0] * len(fens), [1] * len(fens), [0] * len(fens)
    )
    return b

def get_single_activation(model, forward_mode, feature_set, fens):

    model.forward_mode = forward_mode

    b = get_data(feature_set, fens)
    
    (
        us,
        them,
        white_indices,
        white_values,
        black_indices,
        black_values,
        outcome,
        score,
        psqt_indices,
        layer_stack_indices,
    ) = b.contents.get_tensors("cuda")
    out = model.forward(
        us,
        them,
        white_indices,
        white_values,
        black_indices,
        black_values,
        psqt_indices,
        layer_stack_indices,
    )
    out_numpy = out.cpu().detach().numpy()

    del out    


    return out_numpy

In [2]:

features_name = "HalfKAv2_hm^"
feature_set = features.get_feature_set_from_name(features_name)

nnue = M.NNUE(feature_set)
nnue.load_from_checkpoint('/home/ap/nnue-pytorch/logs/lightning_logs/version_19_setting_0/checkpoints/epoch=499-step=3052000.ckpt', feature_set=feature_set)
nnue.cuda()

NNUE(
  (input): DoubleFeatureTransformerSlice()
  (layer_stacks): LayerStacks(
    (l1): Linear(in_features=2048, out_features=128, bias=True)
    (l2): Linear(in_features=128, out_features=128, bias=True)
    (output): Linear(in_features=128, out_features=1, bias=True)
    (relu): ReLU()
  )
)

In [3]:
import pandas as pd
db_name = "stockfish_data_fens_01"
dir_name = f"/media/ap/storage/stockfish_data/{db_name}"

# load df
df = pd.read_pickle(f"{dir_name}/concept_df.pkl")

In [14]:
i = 2
forward_mode = 3
fens = [df.iloc[i].fen]

In [15]:
out = get_single_activation(nnue, forward_mode, feature_set, fens)

In [16]:
out.shape

(1, 128)